In [109]:
!pip install tinyec

Defaulting to user installation because normal site-packages is not writeable


In [110]:
!pip install pycryptodome

Defaulting to user installation because normal site-packages is not writeable


In [111]:
import pandas as pd
import pickle
# Descarga de librerías para la generación de llaves
from tinyec import registry
import secrets
import numpy as np
import hashlib

In [112]:
dataset = pd.read_csv(r'Prosumer_ABC.csv', header= 0, sep = ";" )
dataset.head(5)

,ID,Consumo (0) / Producción (1),Dia,Mes,Año,1,2,3,4,5,...,87,88,89,90,91,92,93,94,95,96
0,ABC,0,2,11,2013,58.00,75.0,65.0,0.08,67.0,...,338.0,369.0,318.00,322.0,345.0,298.00,316.00,299.0,341.00,304.0
1,ABC,0,3,11,2013,0.16,0.1,68.0,69.00,75.0,...,67.0,74.0,61.00,84.0,74.0,67.00,77.00,91.0,0.07,61.0
2,ABC,0,4,11,2013,0.08,66.0,77.0,64.00,84.0,...,114.0,106.0,128.00,511.0,74.0,84.00,59.00,69.0,69.00,71.0
3,ABC,0,5,11,2013,68.00,58.0,95.0,81.00,74.0,...,167.0,202.0,0.53,226.0,256.0,315.00,421.00,66.0,0.09,64.0
4,ABC,0,6,11,2013,77.00,61.0,97.0,65.00,78.0,...,117.0,345.0,351.00,69.0,57.0,0.07,0.07,72.0,56.00,78.0


### ECDSA actual

In [113]:
# algritmo euclideano extendido
# a - grupo
# b - elemento
def alg_euc_ext(a,b):
  if b == 0:
    d = a; x = 1; y =0
    return d,x, y

  x1 = 0; x2 = 1; y1 = 1; y2 = 0

  while b != 0:
    q = int(a/b)
    r = a - q*b
    x = x2 - q*x1
    y = y2 - q*y1
    a = b
    b = r
    x2 =x1
    x1 = x
    y2 = y1
    y1 = y

  d = a; x = x2; y = y2
  # y es inverso de a
  return d,x,y

In [126]:
#Modificación a código:
#https://pycoin.readthedocs.io/en/latest/_modules/pycoin/ecdsa/rfc6979.html#:~:text=%5Bdocs%5D%20def%20deterministic_generate_k%28generator_order%2C%20secret_exponent%2C%20val%2C%20hash_f%3Dhashlib.sha256%29%3A%20%22%22%22%20%3Aparam,%2B%20b%27x00%27%2C%20hash_f%29.digest%28%29%20v%20%3D%20hmac.new%28k%2C%20v%2C%20hash_f%29.digest%28%29
import hashlib
import hmac

if hasattr(1, "bit_length"):
    def bit_length(v):
        return v.bit_length()
else:
    def bit_length(self):
        s = bin(self)  # binary representation:  bin(-37) --> '-0b100101'
        s = s.lstrip('-0b')  # remove leading zeros and minus sign
        return len(s)  # len('100101') --> 6

def deterministic_k(generator_order, secret_exponent, val , hash_f):
    n = generator_order
    bln = bit_length(n)
    order_size = (bln + 7) // 8
    hash_size = hash_f().digest_size
    v = b'\x01' * hash_size
    k = b'\x00' * hash_size
    priv_key = int.to_bytes(secret_exponent, byteorder = 'big', length=order_size)
    #h1 = int(hashlib.sha256(m.encode()).hexdigest(), 16)
    h1 = int(hashlib.sha256(m).hexdigest(), 16)
    h1 = h1.to_bytes(32, 'big')
    k = hmac.new(k, v + b'\x00' + priv_key + h1, hash_f).digest()
    v = hmac.new(k, v, hash_f).digest()
    k = hmac.new(k, v + b'\x01' + priv_key + h1, hash_f).digest()
    v = hmac.new(k, v, hash_f).digest()

    while 1:
        t = bytearray()
        while len(t) < order_size:
            v = hmac.new(k, v, hash_f).digest()
            t.extend(v)
        k1 = int.from_bytes(bytes(t), 'big')
        if k1 >= 1 and k1 < n:
            return k1

In [118]:
def firmado_dsa(curve, m):
  # Generación de clave
  q = curve.field.n
  P = curve.g # element G of prime order q is chosen in E(GF(p))
  x = secrets.randbelow(q) #privkey
  Q = x * P # pubkey
  #h = hashlib.sha1(m)
  h = int(hashlib.sha512(m).hexdigest(),16)
  #Firma digital
  r = 0
  s = 0

  while r == 0 or s == 0:
    #k = secrets.randbelow(q)
    k = deterministic_k(q, x, m , hashlib.sha256)
    kP = k * P
    r = kP.x % q
    l = alg_euc_ext(q, k)[2] # inverso de k
    s = (l*(h + x*r))%q
  return r, s, Q, h

def verificado_dsa(curve, r, s, Q, h):
  #Generaicón de clave
  q = curve.field.n
  P = curve.g

  #Verificación de la firma
  if 1<=r and 1<=s and r<=(q-1) and s<=(q-1):
    w = alg_euc_ext(curve.field.n, s)[2] # invserso s
    u1 = h * w % q
    u2 = r * w % q
    v = (u1*P + u2*Q).x%q
    if v == r:
      print('Se acepta la firma')
    else:
      print('Se rechaza la firma')

  else:
    print('Error: no cumple con 1<=r, s<=q-1')

### Vector de prueba con curva

In [119]:
curve = registry.get_curve('brainpoolP256r1')
# Se convierte la base de datos a bytes para su manejo
m = pickle.dumps(dataset, protocol=4)

r,s,Q,h= firmado_dsa(curve, m)

verificado_dsa(curve, r, s, Q, h)

Se acepta la firma


### ECDSA actual con todo el dataframe

In [120]:
# Firmado y lista con la curva, r, s y Q
def firmado_df(df):
  a = []
  for i in df.index:
    curve = registry.get_curve('brainpoolP256r1')
    m = pickle.dumps(df.iloc[i].to_numpy(), protocol=4)
    r,s,Q,h = firmado_dsa(curve, m)
    a.append([curve, r,s,Q,h])
  return a

In [121]:
# Verificación firma
def ver_df(a):
  for i in range(len(a)):
    verificado_dsa(a[i][0],a[i][1],a[i][2],a[i][3],a[i][4])

In [122]:
ver_df(firmado_df(dataset.iloc[0:5]))

Se acepta la firma
Se acepta la firma
Se acepta la firma
Se acepta la firma
Se acepta la firma


### Comprobación RFC (ECDSA, 256 Bits (Prime Field))

In [102]:
#Modificación a código:
#https://pycoin.readthedocs.io/en/latest/_modules/pycoin/ecdsa/rfc6979.html#:~:text=%5Bdocs%5D%20def%20deterministic_generate_k%28generator_order%2C%20secret_exponent%2C%20val%2C%20hash_f%3Dhashlib.sha256%29%3A%20%22%22%22%20%3Aparam,%2B%20b%27x00%27%2C%20hash_f%29.digest%28%29%20v%20%3D%20hmac.new%28k%2C%20v%2C%20hash_f%29.digest%28%29
import hashlib
import hmac

if hasattr(1, "bit_length"):
    def bit_length(v):
        return v.bit_length()
else:
    def bit_length(self):
        s = bin(self)  # binary representation:  bin(-37) --> '-0b100101'
        s = s.lstrip('-0b')  # remove leading zeros and minus sign
        return len(s)  # len('100101') --> 6

def deterministic_k(generator_order, secret_exponent, val , hash_f):
    n = generator_order
    bln = bit_length(n)
    order_size = (bln + 7) // 8
    hash_size = hash_f().digest_size
    v = b'\x01' * hash_size
    k = b'\x00' * hash_size
    priv_key = int.to_bytes(secret_exponent, byteorder = 'big', length=order_size)
    h1 = int(hashlib.sha256(m.encode()).hexdigest(), 16)
    h1 = h1.to_bytes(32, 'big')
    k = hmac.new(k, v + b'\x00' + priv_key + h1, hash_f).digest()
    v = hmac.new(k, v, hash_f).digest()
    k = hmac.new(k, v + b'\x01' + priv_key + h1, hash_f).digest()
    v = hmac.new(k, v, hash_f).digest()

    while 1:
        t = bytearray()
        while len(t) < order_size:
            v = hmac.new(k, v, hash_f).digest()
            t.extend(v)
        k1 = int.from_bytes(bytes(t), 'big')
        if k1 >= 1 and k1 < n:
            return k1

In [128]:
q = int('0xFFFFFFFF00000000FFFFFFFFFFFFFFFFBCE6FAADA7179E84F3B9CAC2FC632551',0)
x = int('0xC9AFA9D845BA75166B5C215767B1D6934E50C3DB36E89B127B8A622B120F6721',0) # priv key
Ux = int('0x60FED4BA255A9D31C961EB74C6356D68C049B8923B61FA6CE669622E60F29FB6',0)
Uy = int('0x7903FE1008B8BC99A41AE9E95628BC64F2F1B20C2D7E9F5177A3C294D4462299',0)
m = "sample".encode()
curve = registry.get_curve('brainpoolP256r1')
#hash = lambda x: hashlib.sha1(x.encode())
k = '0xA6E3C57DD01ABE90086538398355DD4C3B17AA873382B0F24D6129493D8AAD60'
r = '0xEFD48B2AACB6A8FD1140DD9CD45E81D69D2C877B56AAF991C34D0EA84EAF3716'
s = '0xF7CB1C942D657C41D436C7A1B6E29F65F3E900DBB9AFF4064DC4AB2F843ACDA8'

In [141]:
k_calc = deterministic_k(q, x, m , hashlib.sha256)
print('k = ', k_calc)
if int(k, 0) == k_calc:
    print('mismos valores')

k =  75486370184466523516702714224272210659255809472406410223340475427961162083680
mismos valores


In [142]:
r, s, Q, h = firmado_dsa(curve, m , q, x, Ux, Uy, hash)
print('r:', hex(r), '\n','s:', hex(s))

TypeError: firmado_dsa() takes 2 positional arguments but 7 were given

Experimentos

In [43]:
#pruebas de bytes a int
curve = registry.get_curve('brainpoolP256r1')
#x = secrets.randbelow(curve.field.n)
x = int("0xC9AFA9D845BA75166B5C215767B1D6934E50C3DB36E89B127B8A622B120F6721", 0)
P = curve.g
x * P


(6213929673247522162827900163420077025661042955919036052390098433877096548365, 15559205344702045751121271793715075703169183735854098058101708391254918044411) on "brainpoolP256r1" => y^2 = x^3 + 56698187605326110043627228396178346077120614539475214109386828188763884139993x + 17577232497321838841075697789794520262950426058923084567046852300633325438902 (mod 76884956397045344220809746629001649093037950200943055203735601445031516197751)